In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.dataset as dataset
import lib.network as network
import lib.path as path
import lib.utils as utils

In [3]:
data_all=np.array(sorted(path.construct_path_lists("out","data_")))
label_all=np.array(sorted(path.construct_path_lists("out","labels_")))

In [4]:
net=network.R2N2()

learn_rate 0.1, epochs 1, batch_size 3
creating network...
encoder_network
recurrent_module
decoder_network
loss_function
...network created


In [5]:
i=1
x=utils.to_npy(data_all[i])
y=utils.to_npy(label_all[i])

In [ ]:
states_all=net.get_decoder_state(x,y)

In [ ]:
y_hat=net.predict(x)

In [ ]:
utils.imshow_voxel(y_hat[0])

In [ ]:
plt.plot(states_all[23][0,1,2,3,:])

In [ ]:
plt.plot(states_all[7][0,23])

In [ ]:
utils.imshow_voxel(a[0,:,:,:,0])

In [ ]:
model_dir="out/model_{}_{}_{} {}".format(learning_rate,epoch,batch_size,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
    
# train network
loss_all=[]
acc_all=[]
print("starting training")
for e in range(epoch):
    start_time=time.time()
    loss_epoch=[]
    batch_number=0
    data_batchs,label_batchs=utils.get_batchs(data_all,label_all,batch_size)
    for data,label in zip(data_batchs,label_batchs):
        loss_epoch.append(net.train_step(data,label))
        print("epoch_{:03d}-batch_{:03d}: loss={}".format(e,batch_number,loss_epoch[-1]))
        batch_number+=1
    loss_all.append(loss_epoch)
    net.save("{}/epoch_{:03d}".format(model_dir,e),"loss",loss_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))